# Weighted Quasi Interpolant Spline Approximation for Rainfall Fields

We here provide a quantitative evaluation of a novel approach - the Weighted Quasi Interpolant Spline Approximation (w-QISA) -  in the approximation of observed rain data, in real conditions of sparsity of the observations. For a formal definition of w-QISA and its properties, see [1].

This implementation is not optimized for high performance computing, but wants rather to be as intuitive as possible. This implementation rely on the library for tensor product and LR B-splines proposed at https://github.com/qTipTip/LRSplines, which has the advantage to be very simple and thus accessible to a larger public. For industrial grade performance and a more complete set of tools, see the [GoTools library](https://github.com/SINTEF-Geometry/GoTools) written in C++.

### About the data

In Liguria, observed rainfall data are captured by two different rain gauges networks. Liguria corresponds to a long and narrow strip of land, squeezed between the sea, the Alps and the Apennines mountains. The orography and the closeness to the sea make this area particularly interesting for hydro-meteorological events, frequently characterized by heavy rain due to Atlantic low pressure area, augmented by a secondary low pressure area created by the Ligurian sea (Genova Low). Moreover, the several and small catchments typically cause fast ooding events, and even small rivers exhibit high hydraulic energy due to the quick variation of altitude. The rain gauge network we consider here is owned by the ARPAL team of Regione Liguria, and consists of 143 professional measure stations distributed over the whole region. The measures are acquired every 5-20 minutes, and the stations are connected by GPRS and radio link connection, producing about 2 MB data per day. The resolution of the rain gauges is 0; 2mm while their accuracy is in the range of 2% error threshold.  The data are part of the dataset adopted in [2]. The dataset has been here anonymized by performing an affine transformation on the original coordinates.

### About the model

Let $\mathcal{P}$ be a given point cloud that we aim to approximate through a height field $z=f(x,y)$. Let $\mathbf{p}=(p_x,p_y)$ be a vector of positive integers. Lastly, let
- $\mathbf{x}=[x_1,\ldots,x_{n_x+p_x+1}]$ be a $(p_x+1)$-regular knot vector with boundary knots $x_{p_x+1}=a_1$ and $x_{n_x+1}=b_1$.
- $\mathbf{y}=[y_1,\ldots,y_{n_y+p_y+1}]$  be a $(p_y+1)$-regular knot vector with boundary knots $y_{p_y+1}=a_2$ and $y_{n_y+1}=b_2$.

The *Weighted Quasi Interpolant Spline Approximation* of bi-degree $\mathbf{p}$ to $\mathcal{P}$ on the knot vectors $\mathbf{x}$ and $\mathbf{y}$ is defined by
$$f_w(x,y):=\sum_{i=1}^{n_x}\sum_{j=1}^{n_y}\hat{z}_w(x_i^\ast,y_j^\ast)B[x_i,\ldots,x_{i+p_x+1};y_j,\ldots,y_{j+p_y+1}](x,y)$$
where
$$
x_i^\ast:=\dfrac{x_{i+1}+\ldots+x_{i+p_x}}{p_x}, \quad i=1,\ldots,n_x
$$
and
$$
y_j^\ast:=\dfrac{y_{j+1}+\ldots+y_{j+p_y}}{p_y}, \quad j=1,\ldots,n_y
$$
are the *knot averages* and where
$$
\hat{z}(u,v):=\dfrac{\sum\limits_{(x,y,z)\in\mathcal{P}}z\cdot w(x,y,u,v)}{\sum\limits_{(x,y,z)\in\mathcal{P}} w(x,y,u,v)}
$$
is the *control points estimator* of weight function $w:\mathbb{R}^2\times\mathbb{R}^2\to[0,1]$.

**In a nutshell.** We approximate a pointcloud $\mathcal{P}$ by using a tensor mesh and estimation of the control points by weighted averages.

### Libraries

In [1]:
import numpy as np
import itertools
import math
import statistics

import LRSplines
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

### Variables initialization and parameters setting

In [2]:
#Degree along the x-coordinate:
du = 2
#Degree along the y-coordinate
dv = 2
#Cross-validation tests k-NN weights, for k=1,...,k_max:
k_max = 15
#Cross-validation tests meshes with n inner knots per direction, for n=1,...,n_max:
n_max = 15 #for the sake of simplicity we consider a uniform mesh with same number of knots in each direction

# Minimum of the error distribution for each choice of k and n:
min_err = np.zeros((k_max, n_max))
# Maximum of the error distribution for each choice of k and n:
max_err = np.zeros((k_max, n_max))
# Average of the error distribution for each choice of k and n:
mean_err = np.zeros((k_max, n_max))
# Median of the error distribution for each choice of k and n:
median_err = np.zeros((k_max, n_max))
# Standard deviation of the error distribution for each choice of k and n:
std = np.zeros((k_max, n_max))
# Mean square error of the error distribution for each choice of k and n:
MSE = np.zeros((k_max, n_max))

### Main algorithm

We measure the predictive performances of our technique by performing 5 times a 5-fold cross-validation and then averaging the results (min, max, mean, median, std and MSE). For an exhaustive introduction on K-fold cross validation see Section 7.10.1 in [3].

In [3]:
#For each time cross-validation is performed:
for n_CV in range(1,6):
    #For each couple training-validation set
    for n_set in range(5):
        #For each k(-NN):
        for k in range (1, k_max + 1):
            #For each number of inner knots:
            for n in range(1, n_max + 1):
        
                #POINT CLOUDS
                training_point_cloud = np.loadtxt(
                    "data/CV-" + str(n_CV) + "/training_point_cloud_"+ str(n_set) + ".txt", delimiter=',')
                validation_point_cloud = np.loadtxt(
                    "data/CV-" + str(n_CV) + "/test_point_cloud_"+ str(n_set) + ".txt", delimiter=',')
                point_cloud = np.concatenate((training_point_cloud,validation_point_cloud),axis=0)
                
                #Tensor mesh:
                x_min = min(point_cloud[:,0])
                x_max = max(point_cloud[:,0])
                y_min = min(point_cloud[:,1])
                y_max = max(point_cloud[:,1])
                knots_u = [x_min, x_min] + [x_min+i*(x_max - x_min)/n for i in range(n+1)] + [x_max, x_max]
                knots_v = [y_min, y_min] + [y_min+i*(y_max - y_min)/n for i in range(n+1)] + [y_max, y_max]
                TP = LRSplines.init_tensor_product_LR_spline(du, dv, knots_u, knots_v)
                
                #Knot averages:
                grev_x=[0 for x in range(len(TP.S))]
                grev_y=[0 for y in range(len(TP.S))]
                for i in range(len(TP.S)):
                    b=TP.S[i]
                    b_knot_x=b.knots_u
                    b_knot_y=b.knots_v
                    grev_x[i]=np.sum(b_knot_x[1:du+1])/du
                    grev_y[i]=np.sum(b_knot_y[1:dv+1])/dv
                    
                #Coefficients of k-NN QISA
                for l in range(len(TP.S)):
                    #List of distances to the knot averages:
                    local_distance = [math.sqrt(math.pow(training_point_cloud[m,0]-grev_x[l],2)+
                                                math.pow(training_point_cloud[m,1]-grev_y[l],2)) \
                                      for m in range(training_point_cloud.shape[0])]
                    #Sorted points according to the L2 distance:
                    sorted_indexes = np.argsort(local_distance)
                    sorted_distances = np.sort(local_distance)
                    #k-NN to the Greville's axis:
                    kNN = training_point_cloud[sorted_indexes[0:k]]
                     #Average of the z-component of the k-NN set:
                    TP.S[l].coefficient = np.sum(kNN[:,2])/k
                    
                #Validation error:
                val_err = [0 for l in range(len(validation_point_cloud[:,0]))]
                for l in range(len(validation_point_cloud[:,0])):
                    val_err[l] = abs(TP(validation_point_cloud[l,0], validation_point_cloud[l,1])-validation_point_cloud[l,2])
                #Centered validation error:
                cen_val_err = [0 for l in range(len(validation_point_cloud[:,0]))]
                for l in range(len(validation_point_cloud[:,0])):
                    cen_val_err[l] = val_err[l] - sum(val_err) / len(val_err)
                
                
                #Statistics for the error distribution of a single 5-fold cross validation.
                #Min and max
                min_err[k-1][n-1] = min_err[k-1][n-1] + 1/5 * min(val_err)
                max_err[k-1][n-1] = max_err[k-1][n-1] + 1/5 * max(val_err)
                #Mean and median
                mean_err[k-1][n-1]   = mean_err[k-1][n-1]   + 1/5 * sum(val_err) / len(val_err)
                median_err[k-1][n-1] = median_err[k-1][n-1] + 1/5 * statistics.median(val_err)
                #std and MSE
                std[k-1][n-1] = std[k-1][n-1] + 1/5 * math.sqrt(1/(len(cen_val_err)-1)*sum(map(lambda x:x*x,cen_val_err)))
                MSE[k-1][n-1] = MSE[k-1][n-1] + 1/5 * 1/(len(val_err))*sum(map(lambda x:x*x,val_err))


#Statistics for the error distribution when repeating 5 different times a 5-fold cross validation:
std = std/5
MSE = MSE/5
min_err  = min_err/5
max_err  = max_err/5
mean_err = mean_err/5
median_err = median_err/5

### Statistics for the error distribution


In [4]:
ind_min_MSE = np.unravel_index(np.argmin(MSE, axis=None), MSE.shape)
print("Average min is    " + str(min_err[ind_min_MSE]))
print("Average max is    " + str(max_err[ind_min_MSE]))
print("Average mean is   " + str(mean_err[ind_min_MSE]))
print("Average median is " + str(median_err[ind_min_MSE]))
print("Average std is    " + str(std[ind_min_MSE]))
print("Average MSE is    " + str(MSE[ind_min_MSE]))

Average min is    0.04710540096550549
Average max is    2.889891020683991
Average mean is   0.9970376033520634
Average median is 0.9013234682760833
Average std is    0.7082250055027819
Average MSE is    1.5138957396858879


In [5]:
print("Optimal k is " + str(ind_min_MSE[0]+1))
print("Optimal n is " + str(ind_min_MSE[1]+1))


Optimal k is 9
Optimal n is 10


## References

[1] A. Raffo and S. Biasotti. Weighted Quasi Interpolant Spline Approximations for Point Clouds: Properties and Applications, 2019.

[2] G. Patane', A. Cerri, V. Skytt, S. Pittaluga, S. Biasotti, D. Sobrero, T. Dokken and M. Spagnuolo. Comparing Methods for the Approximation of Rainfall Fields in Environmental Applications. *ISPR Journal of Photogrammetry and Remote Sensing*, 127:57–72, 2017.

[3] T. Hastie, R. Tibshirani and J. Friedman. The Elements of Statistical Learning. Data Mining, Inference, and Prediction. Springer. Second Edition. Corrected 12th printing - Jan 13, 2017. 

## Acknowledgements

This project has received funding from the European Union’s Horizon 2020 research and innovation programme under the Marie Skłodowska-Curie grant agreement No 675789. Dott. S. Biasotti work has been partially supported by the EU ERC Advanced Grant CHANGE, grant agreement No. 694515 and the CNR-IMATI project DIT.AD021.080.001.